# BNP Paribas Securities Services - Exploratory Data Analysis

**Période** : Janvier 2024 - Septembre 2025

---
## 1. Configuration & Connexion à la Base de Données

In [29]:
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuration Plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

# Configuration des chemins
BASE_DIR = Path.home() / 'Desktop/BNP Paribas'
DB_PATH = BASE_DIR / 'Data/Processed/hobart_database.db'

# Connexion à la base
conn = sqlite3.connect(str(DB_PATH))

# Vérifier la connexion
tables_count = pd.read_sql_query("SELECT COUNT(*) as count FROM sqlite_master WHERE type='table';", conn)['count'][0]
views_count = pd.read_sql_query("SELECT COUNT(*) as count FROM sqlite_master WHERE type='view';", conn)['count'][0]

print("="*70)
print("📊 CONNEXION À LA BASE DE DONNÉES HOBART")
print("="*70)
print(f"\n✅ Connexion réussie")
print(f"📂 Emplacement: {DB_PATH}")
print(f"📋 {tables_count} tables disponibles")
print(f"👁️  {views_count} vues créées\n")

📊 CONNEXION À LA BASE DE DONNÉES HOBART

✅ Connexion réussie
📂 Emplacement: /Users/jo/Desktop/BNP Paribas/Data/Processed/hobart_database.db
📋 14 tables disponibles
👁️  8 vues créées



---
## 2. Structure de la Base de Données

### Tables et Vues Disponibles

In [30]:
# Lister les tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)

print("="*70)
print("📋 TABLES PRINCIPALES")
print("="*70)
for table in tables['name'].values:
    if table != 'sqlite_stat1':  # Ignorer la table système
        print(f"  • {table}")

print("\n" + "="*70)

# Lister les vues
views = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='view' ORDER BY name;", conn)

print("👁️  VUES OPTIMISÉES")
print("="*70)
for view in views['name'].values:
    print(f"  • {view}")
    
print("\n")

📋 TABLES PRINCIPALES
  • activity
  • businessline
  • businesslineactivity
  • businesslineprocess
  • category
  • deskbusinesslinelink
  • history_activity
  • history_communication
  • history_sr
  • jur_user
  • label
  • sr
  • srcontact

👁️  VUES OPTIMISÉES
  • activity_history_view
  • complete_view
  • contact_effectiveness_metrics
  • monthly_desk_metrics
  • sr_activity_view
  • sr_history_view
  • sr_lifecycle_summary
  • weekly_sr_distribution




---
## 3. Vue d'Ensemble des Données

### Statistiques Globales

In [31]:
# Statistiques globales
stats = pd.read_sql_query("""
SELECT 
    (SELECT COUNT(*) FROM sr) as total_srs,
    (SELECT COUNT(*) FROM activity) as total_activities,
    (SELECT COUNT(*) FROM srcontact) as total_contacts,
    (SELECT COUNT(*) FROM history_sr) as total_sr_history,
    (SELECT COUNT(DISTINCT CATEGORY_ID) FROM sr) as unique_categories,
    (SELECT COUNT(DISTINCT JUR_DESK_ID) FROM sr) as unique_desks
""", conn)

print("="*70)
print("📊 STATISTIQUES GLOBALES DE LA BASE")
print("="*70)
print(f"\n📋 Service Requests (SRs) : {stats['total_srs'][0]:,}")
print(f"🎯 Activités : {stats['total_activities'][0]:,}")
print(f"📞 Communications (Contacts) : {stats['total_contacts'][0]:,}")
print(f"📜 Historique SR (lignes) : {stats['total_sr_history'][0]:,}")
print(f"\n📂 Catégories uniques : {stats['unique_categories'][0]:,}")
print(f"🏢 Desks actifs : {stats['unique_desks'][0]:,}")
print("\n")

📊 STATISTIQUES GLOBALES DE LA BASE

📋 Service Requests (SRs) : 2,370,662
🎯 Activités : 183,248
📞 Communications (Contacts) : 11,437,378
📜 Historique SR (lignes) : 15,086,275

📂 Catégories uniques : 1,565
🏢 Desks actifs : 877




---
## 4. Période Temporelle Couverte

In [32]:
# Période temporelle
date_range = pd.read_sql_query("""
SELECT 
    MIN(CREATIONDATE) as first_sr,
    MAX(CREATIONDATE) as last_sr,
    COUNT(DISTINCT strftime('%Y-%m', CREATIONDATE)) as unique_months
FROM sr
WHERE CREATIONDATE IS NOT NULL
""", conn)

first_date = pd.to_datetime(date_range['first_sr'][0])
last_date = pd.to_datetime(date_range['last_sr'][0])
months_span = date_range['unique_months'][0]

print("="*70)
print("📅 PÉRIODE TEMPORELLE")
print("="*70)
print(f"\n🗓️  Premier SR : {first_date.strftime('%d %B %Y')}")
print(f"🗓️  Dernier SR : {last_date.strftime('%d %B %Y')}")
print(f"📊 Mois couverts : {months_span} mois")
print(f"⏱️  Durée totale : {(last_date - first_date).days} jours\n")

📅 PÉRIODE TEMPORELLE

🗓️  Premier SR : 03 January 2023
🗓️  Dernier SR : 31 December 2025
📊 Mois couverts : 36 mois
⏱️  Durée totale : 1093 jours



In [33]:
conn = sqlite3.connect(str(DB_PATH))

In [34]:
# Top 50 SRs triés par date de création (plus tôt d'abord)
sr_top50_early = pd.read_sql_query("""
SELECT *
FROM sr
WHERE CREATIONDATE IS NOT NULL
ORDER BY CREATIONDATE ASC
LIMIT 50
""", conn)

sr_top50_early['CREATIONDATE'] = pd.to_datetime(sr_top50_early['CREATIONDATE'])
display(sr_top50_early)

,ID,SRNUMBER,CATEGORY_ID,SUBCATEGORY_ID,JUR_ASSIGNEE_ID,STATUS_ID,EXPECTED_FIRST_RESPONSE_DATE,EXPECTED_ACKNOWLEDGEMENT_DATE,EXPIRATION_DATE,CLOSINGDATE,JUR_DESK_ID,CREATIONDATE,PRIORITY_ID,ROOTSR_ID,TYPE_ID,QUICK_FULFILLMENT_ID,QUICK_ANSWER,DEMAND_DATE,ACKNOWLEDGE_DATE,INTERNAL_FLAG,REOPEN_DATE,ALERT,ISSCHEDULER,UPDATE_DATE,NEXTACTNUMBER,CREATOR_ID,JUR_ASSIGNEDGROUP_ID,ESTIMATEDTIME,PRODUCERUSER_ID,PRODUCERDATE,CHECKERUSER_ID,CHECKERDATE,CALCDUEDATE,HAS_NEW,WORKFLOW_ID,ISSUER,EFFECTIVE_FIRST_RESPONSE_DATE,CONFIRMATION_TO_BE_SENT,FINAL_RESPONSE_DATE,REMINDER_DATE,INITIAL_TASK_ID,SUSPEND_DURATION,TREATMENT_TIME,AUTO_SUGGEST,SR_DRAFT_TYPE,LEAD_TIME_EX1,LEAD_TIME_EX3,LEAD_TIME_EX2,LEAD_TIME_EX4,LEAD_TIME_EX5,IS_MULTICHANNEL_SR,FIRST_ACTION_EMAIL_DATE,REPLY_TIME,ISSUER_TYPE_VALUE
0,11425541,[TAXGCDE-SEB-9363],331418,331419,18011.0,33,None,None,2025-09-26 16:00:00,2025-10-11 07:05:46.570000,74605,2023-01-03 09:06:26.215,1719,None,1729,869492.0,0,2023-01-03 09:05:31.763000,None,0,2025-10-22 19:04:53.719000,0,0,2025-12-15 07:28:49.699000,NaN,26585,74605,None,NaN,None,None,None,None,0,1749,CLIENT,None,0,2025-07-21 08:03:38.036000,2025-09-25 16:00:00,None,None,6638:58:06,None,None,None,None,None,None,6638:57:12,0,2025-06-23 11:00:28.697000,2084920.0,0
1,11430023,[TAXGCDE-SEB-9379],331418,331419,18011.0,4,None,None,2025-09-24 16:00:00,None,74605,2023-01-03 12:39:52.880,1719,None,1729,869492.0,0,2023-01-03 12:39:34.945000,None,0,None,0,0,2025-12-18 09:50:18.286000,NaN,26585,74605,None,NaN,None,None,None,None,0,1749,CLIENT,None,0,None,2025-09-23 16:00:00,None,None,None,None,None,None,None,None,None,2444:20:7,0,2023-10-23 07:21:31.217000,89886.0,0
2,11430297,[TAXGCDE-SEB-9384],331418,331419,18011.0,7,None,None,2025-09-22 16:00:00,2025-12-15 12:41:03.572000,74605,2023-01-03 12:55:44.725,1719,None,1729,869492.0,0,2023-01-03 12:55:22.686000,None,0,None,0,0,2025-12-15 12:41:03.572000,NaN,26585,74605,None,NaN,None,None,None,None,0,1749,CLIENT,None,0,2025-11-04 10:27:17.160000,2025-09-19 16:00:00,None,None,7397:31:54,None,None,None,None,None,None,7397:31:32,0,2025-09-24 13:15:52.360000,1123102.0,0
3,11491128,[TACS-SAN-4073],505685,505686,25379.0,7,2023-01-31 09:58:08,2022-12-21 09:58:08,2026-01-02 17:00:00,2025-12-23 09:44:51.537000,77115,2023-01-10 07:22:55.435,1719,None,1730,966631.0,0,2022-12-20 09:58:08,None,0,2024-04-18 13:06:31,0,0,2025-12-23 09:44:51.537000,NaN,25379,77115,None,NaN,None,None,None,None,0,1749,INTERNAL,None,1,2025-12-23 09:44:51.658000,2025-12-31 17:00:00,None,None,5903:46:43,None,None,124:1:52,5903:46:43,None,None,5779:44:51,0,None,NaN,0
4,11501399,[ITAM3PT-FMA-1604],291371,291387,44965.0,7,2023-11-08 09:52:02.939000,2023-11-08 09:52:02.939000,2025-12-03 18:00:00,2025-12-03 16:53:40.280000,77589,2023-01-10 15:49:16.650,1720,None,1729,969668.0,0,2023-01-10 15:48:36.717000,2023-01-10 16:13:03.711000,0,2025-11-27 14:25:21.221000,0,0,2025-12-03 16:53:40.280000,23.0,9777,77589,None,NaN,None,None,None,None,0,1749,CLIENT,None,1,2025-12-03 12:33:34.944000,2025-12-03 17:00:00,None,None,1264:15:02,None,None,None,None,None,0:23:47,1264:14:22,0,None,NaN,1
5,11509304,[TAXEUROIPT-DEB-13524],172255,214186,35272.0,33,2023-11-28 15:35:28.845000,2023-11-23 15:35:28.845000,2025-06-26 17:00:00,None,75365,2023-01-11 10:39:29.066,1719,None,1730,774412.0,0,2023-01-11 10:39:01.524000,2023-01-11 10:49:42.033000,0,2025-12-04 11:46:51.975000,0,0,2025-12-15 10:04:14.143000,NaN,19052,75365,None,NaN,None,None,None,None,0,1749,INTERNAL,2023-06-20 09:12:49.206000,1,2025-06-12 15:16:19.838000,2025-06-16 17:00:00,None,1305:31:45,None,None,None,None,None,None,0:10:12,None,0,None,NaN,1
6,11513508,[INVBE-SJA-1218],422511,422512,43989.0,7,None,None,2023-01-30 02:30:00,2025-12-30 12:20:58.373000,73905,2023-01-11 14:31:17.726,1719,None,1779,749464.0,0,2023-01-11 14:30:31.071000,None,0,2025-12-30 12:14:21.668000,1,0,2025-12-30 12:20:58.373000,NaN,24302,73905,None,NaN,None,None,None,None,0,1749,INTERNAL,None,0,2025-12-30 12:18:41.499000,2023-01-27 11:30:00,None,No

---
## 5. Distribution des Volumes par Table

In [35]:
# Volumes par table
table_volumes = []
main_tables = ['sr', 'activity', 'srcontact', 'history_sr', 'history_activity', 'history_communication']

for table in main_tables:
    try:
        count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table}", conn)['count'][0]
        table_volumes.append({'Table': table.upper(), 'Lignes': count})
    except:
        pass

df_volumes = pd.DataFrame(table_volumes).sort_values('Lignes', ascending=False)

print("="*70)
print("📊 VOLUME DE DONNÉES PAR TABLE")
print("="*70)
print()
display(df_volumes)

# Visualisation
fig = px.bar(
    df_volumes, 
    x='Table', 
    y='Lignes',
    title='Volume de Lignes par Table Principale',
    text='Lignes',
    color='Lignes',
    color_continuous_scale='Blues'
)

fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig.update_layout(height=500, showlegend=False)
fig.show()

📊 VOLUME DE DONNÉES PAR TABLE



,Table,Lignes
5,HISTORY_COMMUNICATION,30988063
3,HISTORY_SR,15086275
2,SRCONTACT,11437378
0,SR,2370662
4,HISTORY_ACTIVITY,1100518
1,ACTIVITY,183248


---
## 6. Structure des Tables Principales

### Inspection des Colonnes et Types de Données

In [36]:
# Fonction pour afficher la structure d'une table
def show_table_structure(table_name):
    schema = pd.read_sql_query(f"PRAGMA table_info({table_name})", conn)
    
    print(f"\n{'='*70}")
    print(f"📋 TABLE: {table_name.upper()}")
    print(f"{'='*70}\n")
    
    # Afficher les colonnes
    for idx, row in schema.iterrows():
        col_name = row['name']
        col_type = row['type']
        pk = " 🔑 PRIMARY KEY" if row['pk'] == 1 else ""
        notnull = " ⚠️ NOT NULL" if row['notnull'] == 1 else ""
        
        print(f"  • {col_name:<30} {col_type:<15} {pk}{notnull}")
    
    # Compter les lignes
    count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table_name}", conn)['count'][0]
    print(f"\n  📊 Total de lignes : {count:,}\n")

# Afficher la structure des tables principales
for table in ['sr', 'activity', 'srcontact']:
    show_table_structure(table)


📋 TABLE: SR

  • ID                             INTEGER         
  • SRNUMBER                       TEXT            
  • CATEGORY_ID                    INTEGER         
  • SUBCATEGORY_ID                 INTEGER         
  • JUR_ASSIGNEE_ID                REAL            
  • STATUS_ID                      INTEGER         
  • EXPECTED_FIRST_RESPONSE_DATE   TIMESTAMP       
  • EXPECTED_ACKNOWLEDGEMENT_DATE  TIMESTAMP       
  • EXPIRATION_DATE                TIMESTAMP       
  • CLOSINGDATE                    TIMESTAMP       
  • JUR_DESK_ID                    INTEGER         
  • CREATIONDATE                   TIMESTAMP       
  • PRIORITY_ID                    INTEGER         
  • ROOTSR_ID                      REAL            
  • TYPE_ID                        INTEGER         
  • QUICK_FULFILLMENT_ID           REAL            
  • QUICK_ANSWER                   INTEGER         
  • DEMAND_DATE                    TIMESTAMP       
  • ACKNOWLEDGE_DATE               TIMESTAMP      

---
## 7. Échantillons de Données

### Aperçu des Premières Lignes

In [37]:
# Échantillon de la table SR
print("="*70)
print("📋 ÉCHANTILLON: TABLE SR (5 premières lignes)")
print("="*70)
print()

sample_sr = pd.read_sql_query("""
SELECT 
    ID,
    SRNUMBER,
    CATEGORY_ID,
    STATUS_ID,
    PRIORITY_ID,
    CREATIONDATE,
    CLOSINGDATE,
    JUR_DESK_ID
FROM sr 
LIMIT 5
""", conn)

display(sample_sr)

📋 ÉCHANTILLON: TABLE SR (5 premières lignes)



,ID,SRNUMBER,CATEGORY_ID,STATUS_ID,PRIORITY_ID,CREATIONDATE,CLOSINGDATE,JUR_DESK_ID
0,20733499,[PROXYHUB-CES-11257],158224,7,1719,2024-12-16 14:48:59.753000,2025-01-28 07:47:21.360000,71958
1,20973516,[PROXYHUB-VPR-11305],158224,7,1719,2025-01-02 09:48:48.807000,2025-01-03 12:00:44.728000,71958
2,21118903,[PROXYHUB-SER-11337],462499,7,1720,2025-01-10 11:00:46.171000,2025-01-27 09:17:49.133000,71958
3,21314667,[PROXYHUB-VPR-11374],158264,7,1720,2025-01-22 09:35:57.026000,2025-01-22 14:37:46.148000,71958
4,21377448,[PROXYHUB-SER-11386],158224,7,1719,2025-01-24 16:23:54.644000,2025-01-28 07:47:50.595000,71958


In [38]:
# Échantillon de la table ACTIVITY
print("\n" + "="*70)
print("🎯 ÉCHANTILLON: TABLE ACTIVITY (5 premières lignes)")
print("="*70)
print()

sample_activity = pd.read_sql_query("""
SELECT 
    ID,
    SR_ID,
    TYPE_ID,
    STATUS_ID,
    CREATIONDATE,
    CLOSINGDATE
FROM activity 
LIMIT 5
""", conn)

display(sample_activity)


🎯 ÉCHANTILLON: TABLE ACTIVITY (5 premières lignes)



,ID,SR_ID,TYPE_ID,STATUS_ID,CREATIONDATE,CLOSINGDATE
0,1898978,15489299,1579,31,2024-01-09 18:23:40.582000,2024-01-11 11:47:42.569000
1,1889810,15458004,1579,31,2024-01-03 15:55:44.626000,2024-01-03 16:57:22.625000
2,1891187,15470504,1579,31,2024-01-04 11:58:33.645000,2024-01-04 13:26:31.475000
3,1902051,15524639,1579,31,2024-01-11 10:55:02.368000,2024-01-11 11:04:11.110000
4,1889805,15458004,1579,31,2024-01-03 15:55:08.758000,2024-01-04 11:36:50.365000


In [39]:
# Fermer la connexion
conn.close()
print("✅ Connexion fermée")

✅ Connexion fermée


In [40]:
import sqlite3
import pandas as pd
pd.set_option('display.max_columns', None)
conn = sqlite3.connect("../Data/Processed/hobart_database.db")

tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%' ORDER BY name;",
    conn
)["name"].tolist()

for t in tables:
    print(f"\n=== {t} ===")
    df = pd.read_sql_query(f"SELECT * FROM {t} LIMIT 5;", conn)
    display(df)

conn.close()


=== activity ===


,ID,SR_ID,CREATIONDATE,CLOSINGDATE,STATUS_ID,TYPE_ID,IMPORTANCE_ID,NOTIFIED,NOTIFICATIONDATE,EXPIRATIONDATE,RESOLUTION_LABEL_ID,UPDATE_DATE,PRODUCERDATE,CHECKERDATE,ACTNUMBER,CRAFT_ACTIVITY_ID,JUR_ASSIGNEDGROUP_ID,JUR_OWNERGROUP_ID,JUR_OWNER_ID,JUR_ASSIGNEDUSER_ID,PRODUCERUSER_ID,CHECKERUSER_ID,COMPLETIONDATE,WORKFLOW_ID,TASK_NUMBER,IS_DRAFT,LAST_UPDATE_BY_DESK,CREATOR_DESK_ID,CREATOR_USER_ID,IS_ACCEPTED,IS_QUICK_TASK,ACCEPTED_DATE,REJECTED_DATE,CHASER_NUMBER
0,1898978,15489299,2024-01-09 18:23:40.582000,2024-01-11 11:47:42.569000,31,1579,1577,0,2024-01-09 10:00:38,None,None,2024-01-11 11:47:42.569000,None,None,[AGEDINCCLAIMS-FLN-763].1,None,81567,None,None,36692.0,None,None,2024-01-11 11:47:42.569000,1751,1.0,0,81567,81567,8583,1.0,1.0,2024-01-11 11:47:40.677000,None,0
1,1889810,15458004,2024-01-03 15:55:44.626000,2024-01-03 16:57:22.625000,31,1579,1577,0,2024-01-03 18:00:44,None,None,2024-01-03 16:57:22.625000,None,None,[LCCUK1-RAR-164731].4,None,73246,None,None,28525.0,None,None,2024-01-03 16:57:22.625000,1751,4.0,0,73246,73246,10326,1.0,1.0,2024-01-03 16:05:51.346000,None,0
2,1891187,15470504,2024-01-04 11:58:33.645000,2024-01-04 13:26:31.475000,31,1579,1577,0,2024-01-05 09:43:58,None,None,2024-01-04 13:26:31.475000,None,None,[ITAM2PT-NAR-24409].1,None,73152,None,None,26619.0,None,None,2024-01-04 13:26:31.475000,1751,1.0,0,73152,73152,16820,1.0,1.0,2024-01-04 12:52:08.943000,None,0
3,1902051,15524639,2024-01-11 10:55:02.368000,2024-01-11 11:04:11.110000,31,1579,1577,0,2024-01-15 17:00:00,None,None,2024-01-11 11:04:11.110000,None,None,[ITAM1PT-TIA-16510].2,None,73151,None,None,26226.0,None,None,2024-01-11 11:04:11.110000,1751,2.0,0,73151,73151,25041,1.0,1.0,2024-01-11 10:56:16.585000,None,0
4,1889805,15458004,2024-01-03 15:55:08.758000,2024-01-04 11:36:50.365000,31,1579,1577,0,2024-01-03 18:00:44,None,None,2024-01-04 11:36:50.365000,None,None,[LCCUK1-RAR-164731].2,None,73246,None,None,35152.0,None,None,2024-01-04 11:36:50.365000,1751,2.0,0,73246,73246,10326,1.0,1.0,2024-01-03 16:45:29.255000,None,0



=== businessline ===


,ID,NAME
0,1,BP2S



=== businesslineactivity ===


...

,ID,NAME
0,4,ONGOING
1,5,PENDING
2,6,COMPLETED
3,7,CLOSED
4,8,ABANDONED



=== sr ===


,ID,SRNUMBER,CATEGORY_ID,SUBCATEGORY_ID,JUR_ASSIGNEE_ID,STATUS_ID,EXPECTED_FIRST_RESPONSE_DATE,EXPECTED_ACKNOWLEDGEMENT_DATE,EXPIRATION_DATE,CLOSINGDATE,JUR_DESK_ID,CREATIONDATE,PRIORITY_ID,ROOTSR_ID,TYPE_ID,QUICK_FULFILLMENT_ID,QUICK_ANSWER,DEMAND_DATE,ACKNOWLEDGE_DATE,INTERNAL_FLAG,REOPEN_DATE,ALERT,ISSCHEDULER,UPDATE_DATE,NEXTACTNUMBER,CREATOR_ID,JUR_ASSIGNEDGROUP_ID,ESTIMATEDTIME,PRODUCERUSER_ID,PRODUCERDATE,CHECKERUSER_ID,CHECKERDATE,CALCDUEDATE,HAS_NEW,WORKFLOW_ID,ISSUER,EFFECTIVE_FIRST_RESPONSE_DATE,CONFIRMATION_TO_BE_SENT,FINAL_RESPONSE_DATE,REMINDER_DATE,INITIAL_TASK_ID,SUSPEND_DURATION,TREATMENT_TIME,AUTO_SUGGEST,SR_DRAFT_TYPE,LEAD_TIME_EX1,LEAD_TIME_EX3,LEAD_TIME_EX2,LEAD_TIME_EX4,LEAD_TIME_EX5,IS_MULTICHANNEL_SR,FIRST_ACTION_EMAIL_DATE,REPLY_TIME,ISSUER_TYPE_VALUE
0,20733499,[PROXYHUB-CES-11257],158224,158234,26539.0,7,None,None,2024-12-17 12:46:31,2025-01-28 07:47:21.360000,71958,2024-12-16 14:48:59.753000,1719,None,1729,NaN,0,2024-12-16 14:46:31,None,0,None,0,0,2025-01-28 07:47:21.360000,None,29241,71958,None,None,None,None,None,None,0,1749,INTERNAL,None,0,2025-01-21 16:58:20.197000,2024-12-17 11:46:31,None,0:0:3,262:11:49,None,None,0:2:28,262:11:49,None,None,262:09:20,0,None,None,0
1,20973516,[PROXYHUB-VPR-11305],158224,158226,26539.0,7,None,None,2025-01-02 17:48:48.809000,2025-01-03 12:00:44.728000,71958,2025-01-02 09:48:48.807000,1719,None,1779,NaN,0,2025-01-02 09:48:09.015000,None,0,None,0,0,2025-01-03 12:00:44.728000,None,26539,71958,None,None,None,None,None,None,0,1749,THIRD_PARTY,None,0,2025-01-03 11:56:57.728000,2025-01-02 16:48:48.809000,None,None,12:8:47,None,None,None,None,None,None,12:8:8,0,None,None,0
2,21118903,[PROXYHUB-SER-11337],462499,462502,26539.0,7,None,None,2025-01-13 09:00:04,2025-01-27 09:17:49.133000,71958,2025-01-10 11:00:46.171000,1720,None,1730,217355.0,0,2025-01-10 11:00:04,None,0,None,0,0,2025-01-27 09:17:49.133000,None,37513,71958,None,None,None,None,None,None,0,1749,INTERNAL,None,0,2025-01-22 12:21:23.634000,2025-01-13 08:00:04,None,None,81:21:19,None,None,0:0:42,81:21:19,None,None,81:20:36,0,None,None,0
3,21314667,[PROXYHUB-VPR-11374],158264,158280,26539.0,7,None,None,2025-01-22 17:35:03,2025-01-22 14:37:46.148000,71958,2025-01-22 09:35:57.026000,1720,None,1730,1405866.0,0,2025-01-22 09:35:03,None,0,None,0,0,2025-01-22 14:37:46.148000,None,26539,71958,None,None,None,None,None,None,0,1749,THIRD_PARTY,None,0,2025-01-22 14:37:46.151000,2025-01-22 16:35:03,None,None,05:02:43,None,None,00:00:54,05:02:43,None,None,05:01:49,0,None,None,0
4,21377448,[PROXYHUB-SER-11386],158224,483490,26539.0,7,None,None,2025-01-27 14:20:44,2025-01-28 07:47:50.595000,71958,2025-01-24 16:23:54.644000,1719,None,1729,NaN,0,2025-01-24 16:20:44,None,0,None,0,0,2025-01-28 07:47:50.595000,None,37513,71958,None,None,None,None,None,None,0,1749,THIRD_PARTY,None,0,2025-01-27 17:16:25.662000,2025-01-27 13:20:44,None,None,10:55:41,None,None,00:03:10,10:55:41,None,None,10:52:30,0,None,None,0



=== srcontact ===


,ID,SR_ID,CREATIONDATE,SUCCESS,OUTBOUND,MEDIUM_ID,IS_DRAFT,RECEPTION_DATE,INTEGRATION_DATE,IMPORTANCE,HAS_ATTACHMENT,IS_AUTOLINK,IS_LOCKED,IS_DR_REQUESTED,IS_READ,IS_RR_REQUEST,IS_RE_REQUESTED,DATE_TIME_SENT,MAILBOX_ID,EMAIL_TYPE,FIRST_READ_BY,FIRST_READ_DATE,LAST_LINKED_BY,LAST_LINKED_DATE,LAST_REMOVED_BY,LAST_REMOVED_DATE,LAST_ARCHIVED_BY,LAST_ARCHIVED_DATE,JUR_CREATOR_ID,JUR_LOCKED_BY,STATUS,FLAG_LABEL_ID,CONFIRMATION_TO_BE_SENT,EMAIL_CATEGORY,TEMPLATE_APPLIED,IS_FORWARDED,FORWARDING_DATE,FORWARDED_BY,EMAIL_SEND_FOR_APPROVAL_ID,EMAIL_APPROVED_BY_ID,EMAIL_REJECTED_BY_ID,EMAIL_SEND_FOR_APPROVAL_DATE,EMAIL_APPROVED_DATE,EMAIL_REJECTED_DATE,FOUR_EYE_CHECK_ENABLED,SENSITIVITY_ID,AUTO_SUGGEST
0,167465386,19853877,2025-01-30 14:00:10.405000,NaN,1,NaN,0,None,None,Normal,NaN,NaN,NaN,0,1.0,0.0,0,2025-01-30 14:00:11.813000,5739,None,9592.0,2025-01-30 14:00:12.270000,None,None,None,None,None,None,NaN,None,13,None,0,NORMAL,0.0,0.0,None,None,None,None,None,None,None,None,0,22.0,None
1,152610548,19853877,2024-10-23 14:04:10.974452,0.0,0,1721.0,0,2024-10-23 14:03:50,2024-10-23 14:04:10.974452,Normal,1.0,0.0,0.0,0,1.0,0.0,0,2024-10-23 14:03:12,5739,HTML,9592.0,2024-10-24 09:00:57.225000,None,None,None,None,None,None,1095.0,None,6,None,0,NORMAL,0.0,NaN,None,None,None,None,None,None,None,None,0,NaN,TO_ARCHIVE
2,160813386,19853877,2024-12-16 12:34:45.818000,NaN,1,NaN,0,None,None,Normal,NaN,NaN,NaN,0,1.0,0.0,0,2024-12-16 12:34:46.891000,5739,None,9592.0,2024-12-16 12:34:47.389000,None,None,None,None,None,None,NaN,None,13,None,0,NORMAL,0.0,0.0,None,None,None,None,None,None,None,None,0,22.0,None
3,159961424,19853877,2024-12-10 15:10:33.262000,NaN,1,NaN,0,None,None,Normal,NaN,NaN,NaN,0,1.0,0.0,0,2024-12-10 15:10:34.743000,5739,None,9592.0,2024-12-10 15:10:35.419000,None,None,None,None,None,None,NaN,None,13,None,0,NORMAL,0.0,0.0,None,None,None,None,None,None,None,None,0,22.0,None
4,152736543,19853877,2024-10-24 09:03:50.534000,NaN,1,NaN,0,None,None,Normal,0.0,NaN,NaN,0,1.0,0.0,0,2024-10-24 09:03:53.110000,5739,None,9592.0,2024-10-24 09:03:54.245000,None,None,None,None,None,None,NaN,None,13,None,1,ACKNOWLEDGED,0.0,0.0,None,None,None,None,None,None,None,None,0,22.0,None
